In [30]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import sys
import datetime as dt
from pyproj import Proj

In [31]:
sys.path.append('/Users/Admin/Documents/scripts/pyfvcom')
import PyFVCOM as pf

In [3]:
def write_river_namelist(output_file, conf_dict):
    """
    Write an FVCOM river namelist file.

    Parameters
    ----------
    output_file : str, pathlib.Path
        Output file to which to write the river configuration.
    forcing_file : str, pathlib.Path
        File from which FVCOM will read the river forcing data.
    vertical_distribution : str, optional
        Vertical distribution of river input. Defaults to 'uniform'.

    """
    for ri in np.arange(0,len(conf_dict['RIVER_GRID_LOCATION'])):
        namelist = {'NML_RIVER': [pf.preproc.NameListEntry('RIVER_NAME', conf_dict['RIVER_NAME'][ri]),
                                  pf.preproc.NameListEntry('RIVER_FILE', conf_dict['RIVER_FILE']),
                                  pf.preproc.NameListEntry('RIVER_GRID_LOCATION', conf_dict['RIVER_GRID_LOCATION'][ri] + 1, 'd'),
                                  pf.preproc.NameListEntry('RIVER_VERTICAL_DISTRIBUTION', conf_dict['RIVER_VERTICAL_DISTRIBUTION'][ri])]}
        pf.preproc.write_model_namelist(output_file, namelist, mode='a')


In [25]:
ds

<xarray.Dataset> Size: 2MB
Dimensions:      (time: 721, rivers: 234)
Coordinates:
  * time         (time) float32 3kB 5.65e+04 5.651e+04 ... 5.659e+04 5.66e+04
Dimensions without coordinates: rivers
Data variables:
    Itime        (time) int32 3kB ...
    Itime2       (time) int32 3kB ...
    river_flux   (time, rivers) float32 675kB ...
    river_temp   (time, rivers) float32 675kB ...
    river_salt   (time, rivers) float32 675kB ...
    river_names  (rivers) object 2kB ...
Attributes:
    source:       Akvaplan-niva BuildRiver, version 1.4
    history:      Created 2025-03-07 at 16:11 h by root
    description:  River forcing (temperature and runoff) for FVCOM 4.x

In [6]:
# lon, lat, Adamselv
pipe_loc = (26.70032, 70.41662)

In [7]:
pipe_c = 10
pipe_flux = 1
pipe_temp = 12
pipe_salt = 12
pipe_name = "Pipe"

In [8]:
pipe_diameter = 3  # example
pipe_depth = 50  # just took the bottom layer from pipe_deps

outflow_deps = [pipe_depth - pipe_diameter/2, pipe_depth + pipe_diameter/2]
# outflow_deps: [48.5, 51.5]

# Make outflow

In [54]:
start_date = dt.datetime(2013,8,1)
release_length = dt.timedelta(days=90)
total_release = 1  # UNITS?

end_release = start_date + release_length
end_run = dt.datetime(2013,10,30)

# Why do we * here??
outflow = total_release*(release_length/dt.timedelta(hours=1))  # Convert to m3/hr
outflow_m3_per_sec = outflow/3600 # convert m3/hr to m3/sec

In [55]:
outflow_m3_per_sec

0.6

In [56]:
# NB! not really good one, as it crashed
out_file = xr.open_dataset('../run-output/adamselv_v01_0001cold1.nc', decode_times=False)  

In [57]:
node_sig_deps = out_file['siglay_center'].values * out_file['h_center'].values[np.newaxis,:]

x = out_file['x'].values
y = out_file['y'].values
xc = out_file['xc'].values
yc = out_file['yc'].values
tri = np.asarray(out_file['nv'][:] - 1).T

In [58]:
utm33 = Proj(proj = 'utm', zone = 33, ellps = 'WGS84', preserve_units = False)
pl = utm33(pipe_loc[0], pipe_loc[1]) 

pipe_nodes = np.array(np.argmin(np.sqrt((x - pl[0])**2 + (y - pl[1])**2)))
pipe_deps = node_sig_deps[:,pipe_nodes].T

In [59]:
pipe_deps

array([ -0.05493379,  -0.17674372,  -0.32497597,  -0.50521664,
        -0.72416292,  -0.98981047,  -1.31165606,  -1.70090777,
        -2.17068965,  -2.7362195 ,  -3.4149287 ,  -4.22647941,
        -5.19262055,  -6.33680901,  -7.68351213,  -9.25710696,
       -11.08031003, -13.17211771, -15.54532049, -18.20377392,
       -21.13975157, -24.33183557, -27.74386657, -31.32542068,
       -35.01407312, -38.73936757, -42.42802001, -46.00957412,
       -49.42160512, -52.61368912])

In [60]:
outflow_deps 

[48.5, 51.5]

In [61]:
pipe_nodes

array(48824)

In [28]:
pipe_dep_split = []

choose = np.array(np.logical_and(pipe_deps >= -outflow_deps[1], pipe_deps <= -outflow_deps[0]))
print(choose)
this_split = np.zeros(len(choose))
this_split[choose] = 1/np.sum(choose)
pipe_dep_split.append(this_split)

pipe_dep_split = np.asarray(pipe_dep_split)
pipe_dep_split_str = []

for this_pipe in pipe_dep_split:
    this_str = ''
    for this_out in this_pipe:
        this_str = this_str + f'{this_out} '
    pipe_dep_split_str.append(this_str[0:-1])

# Generate an array of datetime values at 3-hour intervals
pipe_dt = np.array([
    start_date + dt.timedelta(hours=i)
    for i in range(0, int((end_run - start_date).total_seconds() / 3600) + 1, 3)
])

# Make flux
pipe_flux = np.asarray(np.ones([pipe_nodes.size, pipe_dt.size])*(outflow_m3_per_sec/pipe_nodes.size)).T


[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False  True False]


In [62]:
pipe_dep_split

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])

**!! Comment: then I just went to the original NML file and manually added the pipe river**

# Add river (pipe)

In [64]:
# open riverdata ds
ds = xr.open_dataset('../input/riverdata.nc', decode_times=False)

In [65]:
nt = ds.sizes['time']
nrivers = ds.sizes['rivers']

In [129]:
pipe_flux=1

In [130]:
# Create a new dataset with the same 'time' dimension but 'rivers' = 1
pipe_ds = xr.Dataset(
    {
        "river_flux": (["time", "rivers"], np.full((nt, 1), pipe_flux)),
        "river_temp": (["time", "rivers"], np.full((nt, 1), pipe_temp)),
        "river_salt": (["time", "rivers"], np.full((nt, 1), pipe_salt)),
        "river_names": (["rivers"], np.array([pipe_name], dtype=object))
    },
    coords={
        "time": ds["time"].values,
        # "rivers": np.array([234])  # Single river index = length of the original rivers
    },
    attrs=ds.attrs  # Copy attributes from the original dataset
)

In [132]:
out = xr.concat([ds, pipe_ds], 'rivers', data_vars='minimal')

In [133]:
# Change dimension order for FVCOM
# out = out[['time', 'Itime', 'Itime2', 'river_flux', 'river_temp', 'river_salt', 'river_names']]

In [136]:
out.isel(rivers=-80)

<xarray.Dataset> Size: 26kB
Dimensions:      (time: 721)
Coordinates:
  * time         (time) float32 3kB 5.65e+04 5.651e+04 ... 5.659e+04 5.66e+04
Data variables:
    Itime        (time) int32 3kB 56505 56505 56505 56505 ... 56594 56594 56595
    Itime2       (time) int32 3kB 0 10800000 21600000 ... 64800000 75600000 0
    river_flux   (time) float64 6kB 0.05429 0.0539 0.05351 ... 0.07752 0.07897
    river_temp   (time) float64 6kB 14.83 14.82 14.81 ... 0.2372 0.2366 0.2353
    river_salt   (time) float64 6kB 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    river_names  object 8B '229.Z-small-96                                   ...
Attributes:
    source:       Akvaplan-niva BuildRiver, version 1.4
    history:      Created 2025-03-07 at 16:11 h by root
    description:  River forcing (temperature and runoff) for FVCOM 4.x

In [137]:
# write an intermediate result just to test run with a new river (without FABM tracer)
out.to_netcdf('riverdata_pipe.nc')

# Add tracer river

In [138]:
# just copy the variable
out['tracer1_c'] = out['river_salt']*0

In [139]:
tracer1_c_data = np.zeros(out['river_salt'].shape)
# fill the last river with constant
tracer1_c_data[:,-1] = 10

# dump to the dataset
out['tracer1_c'].values = tracer1_c_data

In [140]:
out.to_netcdf('riverdata_pipe_tracer.nc')

# Add tracer to Nest

In [141]:
nest = xr.open_dataset('fvcom_nest_adamselv.nc', decode_times=False)

In [142]:
nest

<xarray.Dataset> Size: 148MB
Dimensions:        (time: 2161, node: 112, nele: 110, three: 3, siglay: 30,
                    siglev: 31)
Coordinates:
  * time           (time) float32 9kB 5.65e+04 5.651e+04 ... 5.659e+04 5.66e+04
    siglay         (siglay, node) float32 13kB ...
    siglev         (siglev, node) float32 14kB ...
Dimensions without coordinates: node, nele, three
Data variables: (12/23)
    Itime          (time) int32 9kB ...
    Itime2         (time) int32 9kB ...
    lon            (node) float32 448B ...
    lat            (node) float32 448B ...
    lonc           (nele) float32 440B ...
    latc           (nele) float32 440B ...
    ...             ...
    va             (time, nele) float32 951kB ...
    u              (time, siglay, nele) float32 29MB ...
    v              (time, siglay, nele) float32 29MB ...
    temp           (time, siglay, node) float32 29MB ...
    salinity       (time, siglay, node) float32 29MB ...
    hyw            (time, siglev, node) float32 30MB ...

In [143]:
nest['tracer1_c'] = nest['temp']*0 

In [144]:
nest.to_netcdf('fvcom_nest_adamselv_tracer.nc')

# Restart

In [145]:
rst = xr.open_dataset('adamselv_v01_0001.nc', decode_times=False)

In [146]:
rst['tracer1_c'] = rst['temp']*0 

In [148]:
rst.to_netcdf('adamselv_v01_0001_tracer.nc')